**19/12/2021**
<br>
Bruno Mazzilli González

> # Selección de Variables

Tras el analisis procedemos a seleccionar las variables que son mas importantes para predecir la variable target. Un buen metodo para la selección de variables es la Regularización Lasso. Esto nos permetirá quitar 'ruido' de nuestros datos.
<br>
Lasso es un método de regularización que favorece que algunos de los coeficientes terminen valiendo 0. Esto quiere decir que el modelo va a ignorar algunas de las características predictivas, lo que puede ser considerado un tipo de selección automática de características. El incluir menos características supone un modelo más sencillo de interpretar que puede poner de manifiesto las características más importantes del conjunto de datos. En el caso de que exista cierta correlación entre las características predictivas, Lasso tenderá a escoger una de ellas al azar

### Librerias

In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt # plots
import seaborn as sns # plots
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing 
from sklearn.linear_model import Lasso, LassoCV
import pickle
import warnings
warnings.filterwarnings("ignore")

<br>

### Selección de Variables - Variable Target: Fallecimiento

In [2]:
%%time
c_sev_df = pd.read_parquet('data/c_sev_df_encoded.parquet')
#, dtype={"C_YEAR": "int16","target_c_sev": "int16","C_MNTH": "int16","C_WDAY": "int16","C_HOUR": "int16","V_YEAR": "int16","P_AGE": "int16"}
#del c_sev_df['Unnamed: 0']
c_sev_df.head()

CPU times: user 2.32 s, sys: 512 ms, total: 2.83 s
Wall time: 3.17 s


,C_VEHS_10.0,C_VEHS_11.0,C_VEHS_12.0,C_VEHS_13.0,C_VEHS_14.0,C_VEHS_15.0,C_VEHS_16.0,C_VEHS_17.0,C_VEHS_18.0,C_VEHS_19.0,...,P_USER_2.0,P_USER_4.0,P_USER_5.0,C_YEAR,C_MNTH,C_WDAY,C_HOUR,V_YEAR,P_AGE,target_c_sev
index,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1999,1.0,1.0,20.0,1990.0,41.0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1999,1.0,1.0,20.0,1987.0,19.0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1999,1.0,1.0,20.0,1987.0,20.0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1999,1.0,1.0,8.0,1986.0,46.0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1999,1.0,1.0,17.0,1984.0,28.0,0


In [3]:
c_sev_df.shape

(4996219, 151)

In [4]:
c_sev_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4996219 entries, 0 to 5860380
Data columns (total 151 columns):
 #    Column        Dtype  
---   ------        -----  
 0    C_VEHS_10.0   uint8  
 1    C_VEHS_11.0   uint8  
 2    C_VEHS_12.0   uint8  
 3    C_VEHS_13.0   uint8  
 4    C_VEHS_14.0   uint8  
 5    C_VEHS_15.0   uint8  
 6    C_VEHS_16.0   uint8  
 7    C_VEHS_17.0   uint8  
 8    C_VEHS_18.0   uint8  
 9    C_VEHS_19.0   uint8  
 10   C_VEHS_2.0    uint8  
 11   C_VEHS_20.0   uint8  
 12   C_VEHS_21.0   uint8  
 13   C_VEHS_22.0   uint8  
 14   C_VEHS_23.0   uint8  
 15   C_VEHS_24.0   uint8  
 16   C_VEHS_25.0   uint8  
 17   C_VEHS_26.0   uint8  
 18   C_VEHS_27.0   uint8  
 19   C_VEHS_28.0   uint8  
 20   C_VEHS_29.0   uint8  
 21   C_VEHS_3.0    uint8  
 22   C_VEHS_30.0   uint8  
 23   C_VEHS_31.0   uint8  
 24   C_VEHS_32.0   uint8  
 25   C_VEHS_33.0   uint8  
 26   C_VEHS_34.0   uint8  
 27   C_VEHS_35.0   uint8  
 28   C_VEHS_36.0   uint8  
 29   C_VEHS_37.0 

In [5]:
X = c_sev_df.drop("target_c_sev", axis= 1)

In [6]:
y = c_sev_df["target_c_sev"].values

### Definimos el modelo LASSO:

Como este modelo va a ser utilizado para seleccionar aquellas variables más relevantes especificamos que realice 500 iteraciones y cross validation. Además entrenaremos el modelo con toda la muestra ya que este modelo no será utilizado para futuras predicciones o clasificaciones

In [7]:
lassocv = LassoCV(alphas = None, # no predefinimos los alphas, queremos que los seleccione el propio modelo
                  max_iter = 100, # número máximo de iteraciones = 500
                  normalize = True,
                  n_jobs = -1,
                  selection='random') 

In [ ]:
%%time
# entrenamos el modelo de cross validation con parte de los datos (por motivos de procesamiento)
lassocv.fit(X.iloc[1:1000000,], y[1:1000000,])

In [ ]:
# definimos: lasso function, en el que el alpha será el calculado anteriormente
model_lasso = Lasso(alpha=lassocv.alpha_) 
# entrenamos el modelo lasso con toda la muestra
model_lasso.fit(X[1:1000000,], y[1:1000000,])

Visualizamos los coeficientes resultantes del modelo lasso

In [ ]:
# coefficientes lasso de las variables predictoras:
lasso_coefficients = pd.DataFrame(model_lasso.coef_, set(X), columns = ['Coefficients'])

# término independiente del modelo Lasso
lasso_coefficients.loc['Intercept'] = model_lasso.intercept_
lasso_coefficients

# data frame de los coeficientes Lasso
df_lasso = pd.DataFrame(lasso_coefficients)

# ordenamos las variables en función de sus coeficientes
df_lasso_ordered = df_lasso.sort_values(by = "Coefficients")

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None) # para que muestre todos los valores del df

print(df_lasso_ordered)

In [ ]:
# seleccionamos aquellas variables con coef >  |0.001|
important_variables = lasso_coefficients[(lasso_coefficients['Coefficients'] > 0.001) |
                                         (lasso_coefficients['Coefficients'] < -0.001)]

# visualizamos las variables  y los coeficientes que cumplen este requisito
lasso_coefficients_001 = important_variables.sort_values(['Coefficients'])
lasso_coefficients_001

In [ ]:
lasso_coefficients_001.shape

In [ ]:
final_variables = list(lasso_coefficients_001.index) # seleccionamos las variables que cumplen el criterio coef >|0.01|
final_variables

Guardamos en un data frame los datos de las variables seleccionadas (incluyendo la variable target):

In [ ]:
c_sev_df_final = c_sev_df.loc[:,['C_WTHR_2.0',
 'C_RSUR_7.0',
 'C_TRAF_7.0',
 'V_TYPE_7.0',
 'C_TRAF_10.0',
 'P_PSN_31.0',
 'C_CONF_33.0',
 'C_RSUR_3.0',
 'C_RSUR_6.0',
 'C_MNTH',
 'C_TRAF_4.0',
 'C_TRAF_8.0',
 'V_TYPE_10.0',
 'V_TYPE_21.0',
 'C_TRAF_12.0',
 'C_TRAF_13.0',
 'C_CONF_5.0',
 'C_VEHS_26.0',
 'V_TYPE_18.0',
 'C_VEHS_15.0',
 'C_CONF_25.0',
 'C_RALN_4.0',
 'C_VEHS_14.0',
 'C_VEHS_46.0',
 'C_WTHR_7.0',
 'V_TYPE_9.0',
 'C_VEHS_18.0',
 'P_USER_4.0',
 'C_VEHS_10.0',
 'C_VEHS_11.0',
 'P_PSN_33.0',
 'V_YEAR',
 'C_VEHS_4.0',
 'C_TRAF_5.0',
 'C_VEHS_77.0',
 'P_SAFE_10.0',
 'C_RCFG_3.0',
 'C_RSUR_2.0',
 'C_VEHS_29.0',
 'C_RCFG_9.0',
 'C_VEHS_6.0',
 'C_CONF_22.0',
 'C_VEHS_3.0',
 'C_HOUR',
 'P_SAFE_9.0',
 'P_PSN_96.0',
 'V_TYPE_23.0',
 'P_SEX_2',
 'C_RSUR_8.0',
 'C_VEHS_30.0',
 'C_RCFG_2.0',
 'C_RCFG_8.0',
 'V_TYPE_14.0',
 'V_TYPE_17.0',
 'C_TRAF_6.0',
 'C_VEHS_19.0',
 'C_VEHS_34.0',
 'C_VEHS_20.0',
 'C_TRAF_18.0',
 'C_VEHS_56.0',
 'V_TYPE_8.0',
 'C_VEHS_37.0',
 'C_TRAF_9.0',
 'C_VEHS_27.0',
 'C_RALN_5.0',
 'P_PSN_22.0',
 'C_CONF_2.0',
 'C_RCFG_7.0',
 'C_VEHS_31.0',
 'P_USER_2.0',
 'C_RCFG_5.0',
 'C_WTHR_4.0',
 'C_TRAF_16.0',
 'C_VEHS_2.0',
 'C_VEHS_51.0',
 'C_CONF_41.0',
 'C_CONF_21.0',
 'C_VEHS_24.0',
 'C_CONF_24.0',
 'C_CONF_6.0',
 'C_VEHS_32.0',
 'V_TYPE_6.0',
 'C_WTHR_3.0',
 'C_VEHS_7.0',
 'C_VEHS_71.0',
 'P_SAFE_13.0',
 'P_SAFE_2.0',
 'C_RCFG_10.0',
 'C_RSUR_5.0',
 'C_VEHS_57.0',
 'C_VEHS_58.0',
 'P_PSN_13.0',
 'P_ISEV_2.0',
 'C_RSUR_9.0',
 'C_VEHS_13.0',
 'P_PSN_97.0',
 'C_CONF_23.0',
 'C_YEAR',
 'C_CONF_4.0',
 'C_VEHS_44.0',
 'C_VEHS_35.0',
 'P_PSN_21.0',
 'target_c_sev']] 


c_sev_df_final.reset_index(drop=True, inplace=True)
c_sev_df_final.head(3)

In [ ]:
c_sev_df_final.shape

In [ ]:
#calculamos correlaciones con un subset de los datos (por motivos de procesamiento)
corr = c_sev_df_final.sample(n=1000000,axis=0).corr(method='kendall')
plt.figure(figsize=(25,20))
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True # Para que lo muestre triangular
sns.heatmap(corr, annot = False, mask = mask) # que muestre los coeficientes
c_sev_df_final.columns

In [ ]:
s = corr.unstack()
so = s.sort_values(kind="quicksort")
print(so)

Eliminamos algunas de las variables que tienen alta correlación con otras variables (principalmente porque representan el mismo 'feature'). Tambien eliminamos la columna P_ISEV_3.0 ya que esta variable representa lo mismo que nuestra variable target. La variable P_ISEV indica si se necesitó asistencia medica en ese accidente. Esta variable toma un valor de 3 cuando la persona involucrada en el accidento falleció inmediatamente. Por tanto eliminamos la columna P_ISEV_3.0

In [ ]:
#del c_sev_df_final['P_PSN_13.0']
#del c_sev_df_final['V_TYPE_17.0']
#del c_sev_df_final['P_ISEV_3.0']

In [ ]:
c_sev_df_final.shape

### Guardamos el modelo y los datos seleccionados:

In [ ]:
def save_models(filename, model): # definimos una funcion para guardar los modelos 
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

In [ ]:
save_models('models/lasso_model.pkl', model_lasso)

In [ ]:
# how to load the model from disk:
#loaded_model = pickle.load(open('models/lasso_model.pkl', 'rb'))
#result = loaded_model.score(X_test, Y_test)
#or:
#print(loaded_model)

Guardamos los datos seleccionados en un CSV para poder utilizarlo en los siguientes modelos

In [ ]:
c_sev_df_final.to_parquet('data/c_sev_df_final.parquet', index = False)